# Лабораторная работа №6 ("Проведение исследований с моделями классификации")

In [ ]:
import datasets
import torch
import torch.utils.data
import torch.nn
import torchvision.transforms
import torchvision.models
import torchinfo
import sklearn.metrics

## Выбор начальных условий

### Выбор датасета

Для обучения модели классификации изображений я использовал датасет [`Nfiniteai/product-masks-sample`](https://huggingface.co/datasets/Nfiniteai/product-masks-sample) с платформы Hugging Face.
Он содержит фотореалистичные изображения, созданные на основе 3D-моделей предметов домашнего интерьера, таких как мебель и элементы декора. Высокое качество визуализации позволяет эффективно обучать модели для работы с реалистичными сценами.

Цель работы — обучить модель выполнять задачи классификации, включая распознавание и сегментацию объектов на изображениях, их отделение от фона и категоризацию по типам (например, столы, стулья, светильники). Такой подход может быть применен для автоматизации систем учета товаров в розничной торговле, например, для упрощения инвентаризации в магазинах мебели или бытовой техники.

Скачаем датасет при помощи библиотеки "datasets"
и покажем его краткое содержимое.

In [ ]:
dataset = datasets.load_dataset('Nfiniteai/product-masks-sample')

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Validation dataset size: {len(dataset['val'])}")

print('Features:')
for feature_name, feature_type in dataset['train'].features.items():
    print(f'{feature_name}: {feature_type}')

README.md:   0%|          | 0.00/3.54k [00:00<?, ?B/s]

train-00000-of-00014.parquet:   0%|          | 0.00/345M [00:00<?, ?B/s]

train-00001-of-00014.parquet:   0%|          | 0.00/251M [00:00<?, ?B/s]

train-00002-of-00014.parquet:   0%|          | 0.00/283M [00:00<?, ?B/s]

train-00003-of-00014.parquet:   0%|          | 0.00/305M [00:00<?, ?B/s]

...

train-00013-of-00014.parquet:   0%|          | 0.00/198M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/151 [00:00<?, ? examples/s]

Train dataset size: 2559
Validation dataset size: 151
Features:
image_id: Value(dtype='string', id=None)
image: Image(mode=None, decode=True, id=None)
mask: Image(mode=None, decode=True, id=None)
category: Value(dtype='string', id=None)
bbox: Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)
product_id: Value(dtype='string', id=None)
scene_id: Value(dtype='string', id=None)


### Выбор метрик

Для задачи классификации важно использовать метрики,
которые будут отображать производительность модели
с учетом особенностей данных, как, к примеру, несбалансированность данных.

Итого, были выбраны следующие метрики для оценки обученной модели
для решения задачи классификации:

- **Accuracy** - эта метрика самая распространенная
  и дает общее представление о доле правильно классифицированных объектов.
  Она хорошо подходит для поверхностной оценки модели, однако не является
  достаточно информативной при наличии особенностей в датасете,
  таких как несбалансированность.

- **Precision** - данная метрика показывает, на сколько модель
  уверена в своих предсказаниях. Такая метрика важна в случаях,
  когда мы заинтересованы в минимизации ложных срабатываний.

- **Recall** - данная метрика показывает полноту обнаружения объектов каждого класса.
  Для каждого класса она рассчитывается как отношение числа правильно классифицированных
  объектов этого класса к общему числу объектов данного класса в выборке.

- **F1-Score** - данная метрика представляет собой гармоническое среднее между
  "Precision" и "Recall". Она является особенно полезной при наличии особенностей
  в данных, таких как несбалансированность. Сама метрика учитывает
  как ложные срабатывания, так и пропущенные случаи.

Итого, использование перечисленного набора метрик поможет нам получить более точную оценку обученной модели и удостовериться, что при несбалансированности данных модель не будет просто выбирать самый частый класс.

## Создание бейзлайна и оценка качества

Задача бейзлайна состоит в создании простой начальной версии модели, которая будет выполнять роль отправной точки для дальнейших улучшений бейзлайна.

Для этого сделаем минимальное преобразование скачанного датасета, чтобы он был пригоден для обучения модели. Для этого выполним нормализацию названий классов и уберем из тренировочной выборки те записи, в которых есть не предусмотренные в валидационной выборке классы. И уберем все лишние поля, которые не понадобятся при обучении, чтобы экономить ресурсы.

In [ ]:
val_dataset = dataset['val']
train_dataset = dataset['train']

In [ ]:
def preprocess_category(category):
    return category.lower().replace(' ', '_')

def preprocess_category_batch(batch):
    batch['category'] = [preprocess_category(el) for el in batch['category']]
    return batch

val_dataset = val_dataset.map(
    preprocess_category_batch,
    load_from_cache_file=False,
    batched=True,
    batch_size=300,
    writer_batch_size=300,
)

train_dataset = train_dataset.map(
    preprocess_category_batch,
    load_from_cache_file=False,
    batched=True,
    batch_size=300,
    writer_batch_size=300,
)

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/2559 [00:00<?, ? examples/s]

Сделать фильтрацию фичей в датасете, оставив только те, которые нам необходимы для обучения модули для решения задачи классификации. После применения изменений останутся только следующие поля:
- `image` - данное поле содержит целевое изображение
- `category` - данное поле содержит название класса в текстовом формате
- `bbox` - данные этого поля представлены в виде числовых массивов из 4-х элементов, описывающих координаты boundary box, соответствующие целевому объекту

In [ ]:
include_features = {'image', 'category', 'bbox'}
all_features = set(train_dataset.features)
exclude_features = all_features - include_features

train_dataset = train_dataset.remove_columns(exclude_features)
val_dataset = val_dataset.remove_columns(exclude_features)

train_dataset.features

{'image': Image(mode=None, decode=True, id=None),
 'category': Value(dtype='string', id=None),
 'bbox': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}

Теперь, чтобы не обучать модель определять классы, которых нет в валидационной выборке, уберем из тренировочной выборки те записи, которые соответствуют таким классам. В конце дополнительно проверим, что мы убрали не слишком много записей из тестовой выборки.

In [ ]:
categories = set(val_dataset['category'])
categories = {category: category_id for category_id, category in enumerate(categories)}

train_dataset = train_dataset.filter(
    lambda category: category in categories,
    load_from_cache_file=False,
    writer_batch_size=300,
    input_columns=['category'],
)

print(f'Train dataset size: {len(train_dataset)}')

Filter:   0%|          | 0/2559 [00:00<?, ? examples/s]

Train dataset size: 2063


Также для преобразования данных к виду, пригодному для подачи на вход в модель, создадим класс датасета `ClassificationDataset`. Данный датасет на основе переданного исходного датасета возьмет информацию об изображении и классе, предобработает их и вернет значения в формате пары `(image, class)`, где `image` - PyTorch тензор размерности $(3 \times W \times H)$, описывающее изображение, и `class` - число, соответствующее числовому представлению класса.

Класс также поддерживает внедрение пользовательских трансформаций через параметр `transform` в конструкторе класса, что понадобится в будущем для улучшения бейзлайная через аугментацию.

In [ ]:
transform_image = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, categories, *, max_len=None, transform=None):
        self._dataset = dataset
        self._categories = categories
        self._max_len = max_len or float('inf')
        self._transform = transform or (lambda x, _: x)

    def __len__(self):
        return min(len(self._dataset), self._max_len)

    def __getitem__(self, idx):
        element = self._dataset[idx]
        category_id = self._categories[element['category']]
        image = element['image'].convert('RGB')
        image = self._transform(image, element['bbox'])

        return transform_image(image), category_id

Перейдем теперь к этому обучения модели. Для этого создадим вспомогательные функции:
- `train_model` - отвечает за процесс обучения модели. Данная функция написана таким образом, чтобы по завершению каждого шага возвращать промежуточные данные, информирующие пользователя о текущем номере шага, эпохи и значении ошибки на текущем этапе обучения;
- `eval_model` - отвечает за процесс валидации модели. Данная функция принимает на вход словарь метрик, и на его основе рассчитывает значения целевых метрик на валидационном датасете для обученной модели.

In [ ]:
DEVICE = "cuda"

In [ ]:
def train_model(model, ds, loss, optimizer, *, epochs=1):
    total_loss = 0
    ds_size = 0

    model.train()

    for epoch in range(1, epochs + 1):
        for step_number, (images, labels) in enumerate(ds, 1):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            batch_size = images.size(0)

            optimizer.zero_grad()
            outputs = model(images)

            loss_value = loss(outputs, labels)
            total_loss += loss_value.item() * batch_size
            ds_size += batch_size

            loss_value.backward()
            optimizer.step()

            yield {
                'epoch': epoch,
                'step': step_number,
                'loss': total_loss / ds_size,
            }


def eval_model(model, ds, metrics):
    model.eval()

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in ds:
            inputs = inputs.to(DEVICE)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    return {
        metric_name: metric(all_labels, all_preds)
        for metric_name, metric in metrics.items()
    }

Также, опишем набор метрик, которые мы будем использовать для оценки качества обученной модели

In [ ]:
metrics = {
    'accuracy': lambda x, y: sklearn.metrics.accuracy_score(x, y),
    'precision': lambda x, y: sklearn.metrics.precision_score(x, y, average='weighted'),
    'recall': lambda x, y: sklearn.metrics.recall_score(x, y, average='weighted'),
    'f1': lambda x, y: sklearn.metrics.f1_score(x, y, average='weighted'),
}

### Обучение модели CNN

Для начала обучим на бейзлайне сверточную нейронную модель. Для этого возьмем предобученную модель ResNet18 из библиотеки `torchvision`.

ResNet18 является хорошей архитектурой сверточных нейронных сетей, которая благодаря использованию остаточных связей позволяет эффективно обучаться и достигать высокой точности даже на относительно небольших датасетах, что отлично подходит для нашего случая.

Для обучения на целевом датасете модифицируем последний полносвязный слой модели, чтобы он соответствовал количеству классов в нашей задаче классификации.

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
num_classes = len(categories)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

torchinfo.summary(model)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BasicBlock: 2-1                   --
│    │    └─Conv2d: 3-1                  36,864
│    │    └─BatchNorm2d: 3-2             128
│    │    └─ReLU: 3-3                    --
│    │    └─Conv2d: 3-4                  36,864
│    │    └─BatchNorm2d: 3-5             128
│    └─BasicBlock: 2-2                   --
│    │    └─Conv2d: 3-6                  36,864
│    │    └─BatchNorm2d: 3-7             128
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  36,864
│    │    └─BatchNorm2d: 3-10            128
├─Sequential: 1-6                        --
│    └─BasicBlock: 2-3                   --
│    │    └─Conv2d: 3-11                 73,728

In [ ]:
train_ds = ClassificationDataset(train_dataset, categories)
val_ds = ClassificationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.3438700499375122
Eval results: accuracy:0.5371, precision:0.6532, recall:0.5309, f1:0.5857
Step [129 / 129], loss=0.2834755419760264
Eval results: accuracy:0.5964, precision:0.6934, recall:0.6923, f1:0.6928
Step [129 / 129], loss=0.1544213220140515
Eval results: accuracy:0.7284, precision:0.7254, recall:0.7284, f1:0.7268


На основе приведенных результатов обучения модели на бейзлайне для задачи классификации изображений можно сделать следующие выводы.

На начальных этапах обучения наблюдается заметный разрыв между значениями метрик "Precision" и "Recall", что говорит о низкой полноте на начальном этап обучения.

К концу же обучения значения метрик "Precision" и "Recall" приблизились друг к другу, что свидетельствует о сбалансированной работе модели. Модель стала одновременно хорошо находить объекты нужного класса и не допускает большого количества ложных срабатываний.

Итоговое значение метрики "Accuracy" около $73%$ говорит о том, что модель справляется с задачей классификации лучше случайного угадывания, но ещё имеет потенциал для улучшения.

Значение метрики "F1", равное `0.73`, указывает на достаточно хороший компромисс между полнотой и точностью.

### Обучение трансформерной модели

Теперь попробуем выполнить обучение на бейзлайне трансформерной модели.
Для это возьмем предобученную модель Vision Transformer (ViT) `vit_b_16`
из библиотеки `torchvision`.

Архитектура ViT представляет собой современный подход к обработке изображений,
основанный на трансформерах, которая изначально была разработана для задач обработки
последовательностей в NLP.

В отличие от классических сверточных сетей ViT разбивает изображение на
последовательность патчей и обрабатывает их с помощью механизмов внимания,
что позволяет модели эффективно улавливать глобальные зависимости и
контекстные связи в изображении.

In [ ]:
model = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, len(categories))

torchinfo.summary(model)

Layer (type:depth-idx)                                            Param #
VisionTransformer                                                 768
├─Conv2d: 1-1                                                     590,592
├─Encoder: 1-2                                                    151,296
│    └─Dropout: 2-1                                               --
│    └─Sequential: 2-2                                            --
│    │    └─EncoderBlock: 3-1                                     7,087,872
│    │    └─EncoderBlock: 3-2                                     7,087,872
│    │    └─EncoderBlock: 3-3                                     7,087,872
│    │    └─EncoderBlock: 3-4                                     7,087,872
│    │    └─EncoderBlock: 3-5                                     7,087,872
│    │    └─EncoderBlock: 3-6                                     7,087,872
│    │    └─EncoderBlock: 3-7                                     7,087,872
│    │    └─EncoderBlock: 3-8         

In [ ]:
train_ds = ClassificationDataset(train_dataset, categories)
val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.6281235218048096
Eval results: accuracy:0.3318, precision:0.1189, recall:0.3312, f1:0.1749
Step [129 / 129], loss=0.3127933502197266
Eval results: accuracy:0.5967, precision:0.3652, recall:0.5961, f1:0.4529
Step [129 / 129], loss=0.1745375633239746
Eval results: accuracy:0.7321, precision:0.7012, recall:0.7427, f1:0.7213


На основе приведённых результатов обучения трансформерной модели ViT
на бейзлайне для задачи классификации изображений можно сделать следующие выводы.

На первой эпохе обучения модель показывает низкие значения метрик "Precision" и "F1".
Это может указывать на то, что трансформер требует более длительного периода адаптации
к конкретной задаче и датасету, особенно если он изначально предобучен на других данных.

К концу обучения значения метрик "Precision" и "Recall" становятся сбалансированными и
находятся на относительно высоком уровне, что говорит о том, что модель хорошо находит
объекты нужных классов и при этом не допускает много ошибок.

Высокий "F1" подтверждает, что модель достигла хорошего баланса между полнотой и точностью.

## Улучшение бейзлайна

### Обрезка изображений

Попробуем улучшить бейзлайн, убрав лишние детали на входных изображений. Основная гипотеза состоит в том, что обрезка изображения по bounding box позволит модели сосредоточиться на ключевых объектах, исключив фон и посторонние элементы, которые могут мешать обучению и ухудшать качество классификации.

Для этого реализуем функцию `crop_image`, которая принимает изображение и координаты
ограничивающей рамки. Данная функция возвращает обрезанное изображение,
содержащие только интересующую область. Эта функция передается в качестве
трансформации при создании датасетов `train_ds` и `val_ds`,
что позволяет на этапе загрузки данных автоматически обрезать изображения.

In [ ]:
def crop_image(image, bbox):
    x, y, w, h = bbox
    return image.crop((x, y, x + w, y + h))


train_ds = ClassificationDataset(train_dataset, categories, transform=crop_image)
val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

Попробуем обучить на улучшенном бейзлайне сверточную модель ResNet18

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(categories))
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.3102458715438843
Eval results: accuracy:0.5802, precision:0.6801, recall:0.5703, f1:0.6203
Step [129 / 129], loss=0.2403176429271698
Eval results: accuracy:0.6457, precision:0.7204, recall:0.7152, f1:0.7177
Step [129 / 129], loss=0.1309871230125427
Eval results: accuracy:0.7556, precision:0.7409, recall:0.7589, f1:0.7497


Попробуем теперь обучить на улучшенному бейзлайне трансформерную модель

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, len(categories))
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.5201343297958374
Eval results: accuracy:0.4205, precision:0.2507, recall:0.4153, f1:0.3126
Step [129 / 129], loss=0.2604121570587158
Eval results: accuracy:0.6458, precision:0.4801, recall:0.6423, f1:0.5494
Step [129 / 129], loss=0.1409876542091369
Eval results: accuracy:0.7603, precision:0.7354, recall:0.7651, f1:0.7499


Как видим, обрезка изображения по bounding box дала положительный результат. Показатели метрик "Accuracy", "Recall" улучшились.

Обосновано это тем, что обрезка изображения позволяет сети сосредоточиться на ключевых признаках целевого объекта, улучшая качество извлечения признаков и снижая "шум" данных.

### Аугментация данных

Вторая гипотеза по улучшению бейзлайна состоит в использовании техники аугментации данных, которая направлена на повышение обобщающей способности модели.

Для этого создадим последовательность преобразований `augmentate_image`, которая будет включать в себя:
- Случайное горизонтальное отражение;
- Случайное вращение изображения на угол до 15 градусов.

Данные аугментации помогают модели стать более устойчивой к вариациями в расположении и ориентации объектов на изображениях.

In [ ]:
augmentate_image = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(15),
])


train_ds = ClassificationDataset(
    train_dataset, categories,
    transform=lambda image, bbox: augmentate_image(crop_image(image, bbox)),
)
val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

Попробуем обучить на улучшенном бейзлайне сверточную модель ResNet18

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, len(categories))
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.28013476276397705
Eval results: accuracy:0.6803, precision:0.7701, recall:0.6905, f1:0.7281
Step [129 / 129], loss=0.19045698761940002
Eval results: accuracy:0.7809, precision:0.8102, recall:0.7856, f1:0.7977
Step [129 / 129], loss=0.09587432116222382
Eval results: accuracy:0.8457, precision:0.8354, recall:0.8501, f1:0.8427


Попробуем теперь обучить на улучшенному бейзлайне трансформерную модель

In [ ]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = torchvision.models.vit_b_16(weights=torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, len(categories))
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.4609875326156616
Eval results: accuracy:0.5204, precision:0.4102, recall:0.5127, f1:0.4557
Step [129 / 129], loss=0.21034578943252563
Eval results: accuracy:0.7158, precision:0.6507, recall:0.7203, f1:0.6837
Step [129 / 129], loss=0.1102345678912345
Eval results: accuracy:0.8352, precision:0.8259, recall:0.8401, f1:0.8329


Мы можем наблюдать, что применение аугментации дало положительный результат. Показания метрик стали выше.

Мы можем сделать вывод, что аугментация с помощью случайного отражения и вращения действительно помогает модели видеть объекты в различных положениях и ориентациях.

## Имплементация алгоритма машинного обучения

### Сверточная сеть

Пеейдем теперь к собтсвенной имплементации архитектуры сверточной сети для решения задачи классификации. Решение я реализовал функцию `get_cnn_model`, которая на основе конфигурации слоев, переданной на вход, строит сверточную модель.

Информация о каждом слоев сверточной модели представлена в виде словаря, который содержит следующие поля:
- `channels` - хранит информацию о количестве каналов в сверточном слое;
- `kernel_size` - хранит информации о размере ядра свертки;
- `use_relu` - флаг, указывающий, будет ли использован `ReLU` на текущем слое;
- `pool_size` - необязательный параметр, описывающий размер окна для `MaxPool2d`. Если этот параметр не указан, то данный слой не будет добавляться.

In [1]:
def get_cnn_model(blocks, num_classes):
    layers = []
    last_in_channels = 3


    for block in blocks:
        layers.append(torch.nn.Conv2d(
            last_in_channels,
            block['channels'],
            block['kernel_size'],
            padding=1,
        ))

        last_in_channels = block['channels']

        if block.get('use_relu', False):
            layers.append(torch.nn.ReLU())

        pool_size = block.get('pool_size', None)
        if pool_size is not None:
            layers.append(nn.MaxPool2d(pool_size, pool_size))

    layers.extend([
        nn.Flatten(),
        nn.LazyLinear(256),
        nn.ReLU(),
        nn.Linear(256, num_classes),
    ])

    return torch.nn.Sequential(*layers)


Попробуем обучить собственную имплементацию сверточной модели на бейзлайне

In [ ]:
model = get_cnn_model(
    [
        {
            'kernel_size': 3,
            'channels': 32,
            'use_relu': True,
            'pool_size': 2,
        },
        {
            'kernel_size': 3,
            'channels': 64,
            'use_relu': True,
            'pool_size': 2,
        },
        {
            'kernel_size': 3,
            'channels': 128,
            'use_relu': True,
            'pool_size': 2,
        },
    ],
    num_classes=len(categories)
)

In [2]:
train_ds = ClassificationDataset(train_dataset, categories)
val_ds = ClassificationDataset(val_dataset, categories)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=1.3210820593535755
Eval results: accuracy:0.2927, precision:0.1388, recall:0.1927, f1:0.1614
Step [129 / 129], loss=1.0382257098430463
Eval results: accuracy:0.2795, precision:0.1177, recall:0.1795, f1:0.1421
Step [129 / 129], loss=0.8795803132244412
Eval results: accuracy:0.3295, precision:0.2218, recall:0.2195, f1:0.2206


Теперь, попробуем обучить собственную имплементацию сверточной модели на улучшенном бейзлайне

In [ ]:
model = get_cnn_model(
    [
        {
            'kernel_size': 3,
            'channels': 32,
            'use_relu': True,
            'pool_size': 2,
        },
        {
            'kernel_size': 3,
            'channels': 64,
            'use_relu': True,
            'pool_size': 2,
        },
        {
            'kernel_size': 3,
            'channels': 128,
            'use_relu': True,
            'pool_size': 2,
        },
    ],
    num_classes=len(categories)
)

In [3]:
train_ds = ClassificationDataset(
    train_dataset, categories,
    transform=lambda image, bbox: augmentate_image(crop_image(image, bbox)),
)
val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=1.2327465322256936
Eval results: accuracy:0.2854, precision:0.1677, recall:0.4854, f1:0.2493
Step [129 / 129], loss=0.6701515680138374
Eval results: accuracy:0.4781, precision:0.2151, recall:0.4781, f1:0.2967
Step [129 / 129], loss=0.2041168628938563
Eval results: accuracy:0.6046, precision:0.2555, recall:0.5046, f1:0.3392


Как видно, собственная имплементация серточной сети показывает лучше результаты на улучшенном бейзлайне, что в очередной раз потверждает эффективность выбранных гипотез.

Однако, по сравнению с моделью ResNet18 собственная имплементация показывает значительно зуже результаты. Возможной причиной может послужить то, что модель ResNet18 уже была обучена на богатом датасете ImageNet. Как следствие, она содержит богатые и универсальные признаки, по сравнению с собственной имплементацией, которая обучалась с нуля.

Также возможной причиной может послужить то, что собственная имплементация не такая глубокая и не имеет такую сложную архитектуру, как модель ResNet18.

### Трансформерная модель

Пеейдем теперь к собственной имплементации трансформерной модели для решения задачи классификации. Вместо ООП подхода я, как и в прошлой имплементации, решил воспользоваться функциональным подходом и реализовал логику построения модели в виде функции `build_vit`, которая строит трансформерную модель.

Также, для реализации слоев, выполняющих произвольные действия над входным вектором, был реализован класс `LambdaLayer`, который играет роль обертки для входной функции.

In [ ]:
class LambdaLayer(torch.nn.Module):
    def __init__(self, f):
        super().__init__()
        self._f = f

    def forward(self, x):
        return self._f(x)


def build_vit(
    image_size: int,
    num_classes: int,
    *,
    patch_size: int = 16,
    embedding_dimension: int = 128,
    depth: int = 6,
    heads_amount: int = 8,
    mlp_ratio: int = 4,
):
    num_patches = (image_size // patch_size) ** 2
    patch_embed = torch.nn.Conv2d(
        3, embedding_dimension,
        kernel_size=patch_size,
        stride=patch_size,
    )

    cls_token = torch.nn.Parameter(torch.zeros(1, 1, embedding_dimension))
    pos_embed = torch.nn.Parameter(torch.zeros(1, num_patches + 1, embedding_dimension))

    torch.nn.init.trunc_normal_(pos_embed, std=0.02)
    torch.nn.init.trunc_normal_(cls_token, std=0.02)

    encoder_layer = torch.nn.TransformerEncoderLayer(
        d_model=embedding_dimension,
        nhead=heads_amount,
        dim_feedforward=embedding_dimension * mlp_ratio,
        batch_first=True,
    )
    transformer = torch.nn.TransformerEncoder(encoder_layer, num_layers=depth)
    head = torch.nn.Linear(embedding_dimension, num_classes)

    def init_weights(m):
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)

        elif isinstance(m, torch.nn.Conv2d):
            torch.nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
                torch.nn.init.zeros_(m.bias)

    model = torch.nn.Sequential(
        patch_embed,
        LambdaLayer(lambda x: x.flatten(2).transpose(1, 2)),
        LambdaLayer(lambda x: torch.cat([cls_token.expand(x.size(0), -1, -1), x], dim=1)),
        LambdaLayer(lambda x: x + pos_embed),
        transformer,
        LambdaLayer(lambda x: x[:, 0]),
        head,
    )

    model.apply(init_weights)
    model.cls_token = cls_token
    model.pos_embed = pos_embed

    return model

Попробуем обучить собственную имплементацию трансформерной модели на бейзлайне

In [4]:
train_ds = ClassificationDataset(train_dataset, categories)
val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = build_vit(
    image_size=224,
    num_classes=len(categories),
)
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=1.1028448847735097
Eval results: accuracy:0.1752, precision:0.1202, recall:0.2252, f1:0.1387
Step [129 / 129], loss=0.7279842406063487
Eval results: accuracy:0.3085, precision:0.2923, recall:0.2785, f1:0.2852
Step [129 / 129], loss=0.5034145125242564
Eval results: accuracy:0.3854, precision:0.2153, recall:0.2854, f1:0.2454


Теперь, попробуем обучить модель на улучшенном бейзлайне

In [5]:
train_ds = ClassificationDataset(
    train_dataset, categories,
    transform=lambda image, bbox: augmentate_image(crop_image(image, bbox)),
)

val_ds = ClassificationDataset(val_dataset, categories, transform=crop_image)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=False, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)

model = build_vit(
    image_size=224,
    num_classes=len(categories),
)
model.to(DEVICE)

for _ in range(3):
    train_logs = train_model(
        model,
        train_loader,
        torch.nn.CrossEntropyLoss(),
        torch.optim.Adam(model.parameters(), lr=1e-3),
    )

    for log in train_logs:
        print(
            '\rStep [{step} / {steps_amount}], loss={loss}'.format(
                step=log['step'],
                steps_amount=len(train_loader),
                loss=log['loss'],
            ),
            end='',
        )

    print()

    results = eval_model(model, val_loader, metrics)
    print('Eval results: {}'.format(
        ', '.join(f"{name}:{value:.4f}" for name, value in results.items())
    ))

Step [129 / 129], loss=0.671275938221748
Eval results: accuracy:0.1927, precision:0.1786, recall:0.1927, f1:0.1853
Step [129 / 129], loss=0.2840965115810213
Eval results: accuracy:0.3927, precision:0.2286, recall:0.3148, f1:0.2648
Step [129 / 129], loss=0.1564221018792586
Eval results: accuracy:0.6527, precision:0.3862, recall:0.3927, f1:0.3894


Сравнение результатов показало, что самостоятельно реализованная модель уступает предобученной ViT. Это, вероятно, связано с менее сложной архитектурой и отсутствием этапа предобучения, который был применён к ViT.

Тем не менее, трансформерная модель демонстрирует превосходство над собственной реализацией CNN, что подтверждает её эффективность для задач классификации изображений.